In [11]:
import openai
import os
import uuid

from datetime import datetime
from dotenv import find_dotenv, load_dotenv
from pydantic import BaseModel, Field
from trustcall import create_extractor
from typing import Literal, Optional, TypedDict

from langchain_core.messages import merge_message_runs, HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

In [5]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(llm="gpt-3.5-turbo")

In [7]:
# inspect the tool calls made by Trustcall
class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        # collect information about the tool calls made by the extractor
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

In [8]:
def extract_tool_info(tool_calls, schema_name="Memory"):
    """Extract information from tool calls for both patches and new memories.
    
    Args:
        tool_calls: List of tool calls from the llm
        schema_name: Name of the schema tool (e.g., "Memory", "ToDo", "Profile")
    """

    # initialize list of changes
    changes = []
    
    for call_group in tool_calls:
        for call in call_group:
            if call['name'] == 'PatchDoc':
                changes.append({
                    'type': 'update',
                    'doc_id': call['args']['json_doc_id'],
                    'planned_edits': call['args']['planned_edits'],
                    'value': call['args']['patches'][0]['value']
                })
            elif call['name'] == schema_name:
                changes.append({
                    'type': 'new',
                    'value': call['args']
                })

    # format results as a single string
    result_parts = []
    for change in changes:
        if change['type'] == 'update':
            result_parts.append(
                f"Document {change['doc_id']} updated:\n"
                f"Plan: {change['planned_edits']}\n"
                f"Added content: {change['value']}"
            )
        else:
            result_parts.append(
                f"New {schema_name} created:\n"
                f"Content: {change['value']}"
            )
    
    return "\n\n".join(result_parts)

In [9]:
# update memory tool
class UpdateMemory(TypedDict):
    """Decision on what memory type to update.
    """
    update_type: Literal['user', 'todo', 'instructions']

In [13]:
# user profile schema
class Profile(BaseModel):
    """This is the profile of the user you are chatting with.
    """
    name: Optional[str] = Field(description="The user's name", default=None)
    location: Optional[str] = Field(description="The user's location", default=None)
    job: Optional[str] = Field(description="The user's job", default=None)
    connections: list[str] = Field(
        description="Personal connection of the user, such as family members, friends, or coworkers",
        default_factory=list
    )
    interests: list[str] = Field(
        description="Interests that the user has", 
        default_factory=list
    )

In [ ]:
# ToDo schema
class ToDo(BaseModel):
    task: str = Field(description="The task to be completed.")
    time_to_complete: Optional[int] = Field(description="Estimated time to complete the task (minutes).")
    deadline: Optional[datetime] = Field(
        description="When the task needs to be completed by (if applicable)",
        default=None
    )
    solutions: list[str] = Field(
        description="List of specific, actionable solutions (e.g., specific ideas, service providers, or concrete options relevant to completing the task)",
        min_items=1,
        default_factory=list
    )
    status: Literal["not started", "in progress", "done", "archived"] = Field(
        description="Current status of the task",
        default="not started"
    )

In [14]:
# ceate the Trustcall extractor for updating the user profile 
profile_extractor = create_extractor(
    llm,
    tools=[Profile],
    tool_choice="Profile",
)

# chatbot instruction for choosing what to update and what tools to call 
MODEL_SYSTEM_MESSAGE = """
    You are a helpful chatbot. 

    You are designed to be a companion to a user, helping them keep track of their ToDo list.

    You have a long term memory which keeps track of three things:
    1. The user's profile (general information about them) 
    2. The user's ToDo list
    3. General instructions for updating the ToDo list

    Here is the current User Profile (may be empty if no information has been collected yet):
    <user_profile>
    {user_profile}
    </user_profile>

    Here is the current ToDo List (may be empty if no tasks have been added yet):
    <todo>
    {todo}
    </todo>

    Here are the current user-specified preferences for updating the ToDo list (may be empty if no preferences have been specified yet):
    <instructions>
    {instructions}
    </instructions>

    Here are your instructions for reasoning about the user's messages:

    1. Reason carefully about the user's messages as presented below. 

    2. Decide whether any of the your long-term memory should be updated:
        - If personal information was provided about the user, update the user's profile by calling UpdateMemory tool with type `user`
        - If tasks are mentioned, update the ToDo list by calling UpdateMemory tool with type `todo`
        - If the user has specified preferences for how to update the ToDo list, update the instructions by calling UpdateMemory tool with type `instructions`

    3. Tell the user that you have updated your memory, if appropriate:
        - Do not tell the user you have updated the user's profile
        - Tell the user them when you update the todo list
        - Do not tell the user that you have updated instructions

    4. Err on the side of updating the todo list. No need to ask for explicit permission.

    5. Respond naturally to user user after a tool call was made to save memories, or if no tool call was made.
"""

# Trustcall instruction
TRUSTCALL_INSTRUCTION = """
    Reflect on following interaction. 

    Use the provided tools to retain any necessary memories about the user. 

    Use parallel tool calling to handle updates and insertions simultaneously.

    System Time: {time}
"""

# instructions for updating the ToDo list
CREATE_INSTRUCTIONS = """
    Reflect on the following interaction.

    Based on this interaction, update your instructions for how to update ToDo list items. 

    Use any feedback from the user to update how they like to have items added, etc.

    Your current instructions are:

    <current_instructions>
    {current_instructions}
    </current_instructions>
"""

In [15]:
def task_mAIstro(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Load memories from the store and use them to personalize the chatbot's response.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("profile", user_id)
    memories = store.search(namespace)
    if memories:
        user_profile = memories[0].value
    else:
        user_profile = None

    # retrieve task memory from the store
    namespace = ("todo", user_id)
    memories = store.search(namespace)
    todo = "\n".join(f"{mem.value}" for mem in memories)

    # retrieve custom instructions
    namespace = ("instructions", user_id)
    memories = store.search(namespace)
    if memories:
        instructions = memories[0].value
    else:
        instructions = ""
    
    system_msg = MODEL_SYSTEM_MESSAGE.format(user_profile=user_profile, todo=todo, instructions=instructions)

    # respond using memory as well as the chat history
    response = llm.bind_tools([UpdateMemory], parallel_tool_calls=False).invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": [response]}

def update_profile(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and update the memory collection.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("profile", user_id)
    existing_items = store.search(namespace)

    # format the existing memories for the Trustcall extractor
    tool_name = "Profile"
    existing_memories = (
        [
            (existing_item.key, tool_name, existing_item.value)
            for existing_item in existing_items
        ]
        if existing_items
        else None
    )

    # merge the chat history and the instruction
    TRUSTCALL_INSTRUCTION_FORMATTED=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat())
    updated_messages=list(merge_message_runs(messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]))

    result = profile_extractor.invoke({"messages": updated_messages, 
                                         "existing": existing_memories})

    # save the memories from Trustcall to the store
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        store.put(namespace,
                  rmeta.get("json_doc_id", str(uuid.uuid4())),
                  r.model_dump(mode="json"),
            )
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "updated profile", "tool_call_id":tool_calls[0]['id']}]}

def update_todos(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and update the memory collection.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("todo", user_id)
    existing_items = store.search(namespace)

    # format the existing memories for the Trustcall extractor
    tool_name = "ToDo"
    existing_memories = ([(existing_item.key, tool_name, existing_item.value)
                          for existing_item in existing_items]
                          if existing_items
                          else None
                        )

    # merge the chat history and the instruction
    TRUSTCALL_INSTRUCTION_FORMATTED=TRUSTCALL_INSTRUCTION.format(time=datetime.now().isoformat())
    updated_messages=list(merge_message_runs(messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION_FORMATTED)] + state["messages"][:-1]))

    # initialize the spy for visibility into the tool calls made by Trustcall
    spy = Spy()
    
    # create the Trustcall extractor for updating the ToDo list 
    todo_extractor = create_extractor(
        llm,
        tools=[ToDo],
        tool_choice=tool_name,
        enable_inserts=True
        ).with_listeners(on_end=spy)

    result = todo_extractor.invoke({"messages": updated_messages, 
                                    "existing": existing_memories})

    # save the memories from Trustcall to the store
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        store.put(namespace,
                  rmeta.get("json_doc_id", str(uuid.uuid4())),
                  r.model_dump(mode="json"),
            )
        
    # respond to the tool call made in task_mAIstro, confirming the update
    tool_calls = state['messages'][-1].tool_calls

    # extract the changes made by Trustcall and add the the ToolMessage returned to task_mAIstro
    todo_update_msg = extract_tool_info(spy.called_tools, tool_name)
    return {"messages": [{"role": "tool", "content": todo_update_msg, "tool_call_id":tool_calls[0]['id']}]}

def update_instructions(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and update the memory collection.
    """
    
    user_id = config["configurable"]["user_id"]    
    namespace = ("instructions", user_id)
    existing_memory = store.get(namespace, "user_instructions")
    
    system_msg = CREATE_INSTRUCTIONS.format(current_instructions=existing_memory.value if existing_memory else None)
    
    new_memory = llm.invoke([SystemMessage(content=system_msg)]+state['messages'][:-1] + [HumanMessage(content="Please update the instructions based on the conversation")])

    # overwrite the existing memory in the store 
    key = "user_instructions"
    store.put(namespace, key, {"memory": new_memory.content})
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "updated instructions", "tool_call_id":tool_calls[0]['id']}]}

# conditional edge
def route_message(state: MessagesState, config: RunnableConfig, store: BaseStore) -> Literal[END, "update_todos", "update_instructions", "update_profile"]:
    """Reflect on the memories and chat history to decide whether to update the memory collection.
    """
    message = state['messages'][-1]
    if len(message.tool_calls)==0:
        return END
    else:
        tool_call = message.tool_calls[0]
        if tool_call['args']['update_type'] == "user":
            return "update_profile"
        elif tool_call['args']['update_type'] == "todo":
            return "update_todos"
        elif tool_call['args']['update_type'] == "instructions":
            return "update_instructions"
        else:
            raise ValueError

In [16]:
graph = StateGraph(MessagesState)
graph.add_node(task_mAIstro)
graph.add_node(update_todos)
graph.add_node(update_profile)
graph.add_node(update_instructions)
graph.add_edge(START, "task_mAIstro")
graph.add_conditional_edges("task_mAIstro", route_message)
graph.add_edge("update_todos", "task_mAIstro")
graph.add_edge("update_profile", "task_mAIstro")
graph.add_edge("update_instructions", "task_mAIstro")

across_thread_memory = InMemoryStore()
within_thread_memory = MemorySaver()
graph = graph.compile(checkpointer=within_thread_memory, store=across_thread_memory)

In [17]:
config = {"configurable": {"thread_id": "1", "user_id": "Yoona"}}

input_messages = [HumanMessage(content="My name is Yoona. I live in south korea.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

My name is Yoona. I live in south korea.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_Dybl5D4uv22wOeJyiRjq1zU0)
 Call ID: call_Dybl5D4uv22wOeJyiRjq1zU0
  Args:
    update_type: user
================================= Tool Message =================================

updated profile
================================== Ai Message ==================================

Nice to meet you, Yoona! I've updated your profile with the information you provided. How can I assist you today?


In [18]:
# user input for a ToDo
input_messages = [HumanMessage(content="I have task to learn about MLFlow")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I have task to learn about MLFlow
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_GHZcy1DHg19LrxAub196g1fZ)
 Call ID: call_GHZcy1DHg19LrxAub196g1fZ
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Learn about MLFlow', 'time_to_complete': 120, 'status': 'not started'}
================================== Ai Message ==================================

I have added a new task for you to learn about MLFlow. Let me know if there's anything else you'd like to add or update on your ToDo list.


In [19]:
# user input to update instructions for creating ToDos
input_messages = [HumanMessage(content="When creating or updating ToDo items, include link of reference.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

When creating or updating ToDo items, include link of reference.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_Axjifkden8V3FrbYYy1mofKD)
 Call ID: call_Axjifkden8V3FrbYYy1mofKD
  Args:
    update_type: instructions
================================= Tool Message =================================

updated instructions
================================== Ai Message ==================================

I've noted your preference to include a link to the reference when creating or updating ToDo items. Feel free to provide any references you'd like to include for your tasks. How can I assist you further today?


In [20]:
# check for updated instructions
user_id = "Yoona"

for memory in across_thread_memory.search(("instructions", user_id)):
    print(memory.value)

{'memory': '<current_instructions>\nWhen adding or updating ToDo list items, include a link to the reference or resource related to the task.\n</current_instructions>'}


In [21]:
# user input for a ToDo
input_messages = [HumanMessage(content="I need to learn about fine-tuning model.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I need to learn about fine-tuning model.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_z2BSjc10svm1pRTyuMmQKFj0)
 Call ID: call_z2BSjc10svm1pRTyuMmQKFj0
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Learn about fine-tuning model', 'time_to_complete': 120, 'status': 'not started'}
================================== Ai Message ==================================

I have added the task to learn about fine-tuning models to your ToDo list. If you have any references or resources to include, feel free to share them. Is there anything else you would like to add or update on your list?


In [22]:
# namespace for the memory to save
user_id = "Yoona"
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

{'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': None, 'solutions': [], 'status': 'not started'}
{'task': 'Learn about fine-tuning model', 'time_to_complete': 120, 'deadline': None, 'solutions': [], 'status': 'not started'}


In [23]:
# user input to update an existing ToDo
input_messages = [HumanMessage(content="For the learning MLFlow, i need to get that done max by end of February 2025.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

For the learning MLFlow, i need to get that done max by end of February 2025.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_u6fVpesfI4blWV6aF0HRpGgA)
 Call ID: call_u6fVpesfI4blWV6aF0HRpGgA
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': '2025-02-28T23:59:59', 'status': 'not started'}
================================== Ai Message ==================================

I have updated the task to learn about MLFlow with a deadline of February 28, 2025. This will help you stay on track with your goal. If there's anything else you'd like to adjust or add, feel free to let me know!


In [24]:
# user input for a ToDo
input_messages = [HumanMessage(content="Need to learning for GCP certification too!")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Need to learning for GCP certification too!
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_nmCfSko0E45Mbbp3S5wRCzpg)
 Call ID: call_nmCfSko0E45Mbbp3S5wRCzpg
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Prepare for GCP certification', 'status': 'not started'}

New ToDo created:
Content: {'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': '2025-02-28T23:59:59', 'status': 'not started'}
================================== Ai Message ==================================

I have added the task to prepare for GCP certification to your ToDo list. It's great to see you planning ahead for your learning goals. If there's anything else you'd like to add or update, feel free to share!


In [25]:
# namespace for the memory to save
user_id = "Yoona"
for memory in across_thread_memory.search(("todo", user_id)):
    print(memory.value)

{'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': None, 'solutions': [], 'status': 'not started'}
{'task': 'Learn about fine-tuning model', 'time_to_complete': 120, 'deadline': None, 'solutions': [], 'status': 'not started'}
{'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': '2025-02-28T23:59:59', 'solutions': [], 'status': 'not started'}
{'task': 'Prepare for GCP certification', 'time_to_complete': 180, 'deadline': None, 'solutions': [], 'status': 'not started'}
{'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': '2025-02-28T23:59:59', 'solutions': [], 'status': 'not started'}


In [27]:
config = {"configurable": {"thread_id": "2", "user_id": "Yoona"}}

# chat with the chatbot
input_messages = [HumanMessage(content="I have one month, what tasks can I get done?")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I have one month, what tasks can I get done?
================================== Ai Message ==================================

It seems like you're looking to plan your tasks for the next month. Let's review your current ToDo list to see which tasks you can get done within the next month.
Tool Calls:
  UpdateMemory (call_dfnyEQOQQBXkuKWsHmpLbLTk)
 Call ID: call_dfnyEQOQQBXkuKWsHmpLbLTk
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Learn about MLFlow', 'time_to_complete': 120, 'deadline': '2025-01-31T23:59:59', 'status': 'not started'}
================================== Ai Message ==================================

I have updated your ToDo list. You now have a task to "Learn about MLFlow" with a deadline of January 31st. Good luck with your tasks for the next month! If you need any help or guidance, feel